<img style="float: left;" src="./images/fanniemae.png">

___________

# Mortgage Loan Default Classifier

### Problem Statement:
- Predict whether a mortgage loan will default using Fannie Mae Loan Performance data set from previous quarter
___________

-----------

## Load Datasets and Basic Clean
___

In [19]:
# import necessary packages
import pandas as pd
import numpy as np

In [2]:
# define function to to allow user to assign train year and quarter of training data

In [3]:
# define function to load data set

def load_dataset(year=2005, quarter=1):

    # load acquisition data dictionary csv for column names
    acq_data_dict_fp = './data/acquisition_data_dict_summary.csv'
    list_acq_col_names = pd.read_csv(acq_data_dict_fp)['Field Name'].tolist()
    
    # load acquisition data dictionary csv for column names
    perf_data_dict_fp = './data/performance_data_dict_summary.csv'
    list_perf_col_names = pd.read_csv(perf_data_dict_fp)['Field Name'].tolist()

    # load acquisition csv
    ## string for fp needs variable
    acq_data_fp = f'./data/2005Q1/Acquisition_{year}Q{quarter}.txt'
    df_acq = pd.read_csv(acq_data_fp, sep='|', header=None, names=list_acq_col_names)
    
    # load performance csv
    perf_data_fp = f'./data/2005Q1/Performance_{year}Q{quarter}.txt'
    df_perf = pd.read_csv(perf_data_fp, sep='|', header=None, names=list_perf_col_names)
    
    return df_acq, df_perf, year, quarter

In [4]:
df_acq, df_perf, year, quarter = load_dataset()

### Explore Acquisition File

In [5]:
df_acq.head()

,LOAN IDENTIFIER,ORIGINATION CHANNEL,SELLER NAME,ORIGINAL INTEREST RATE,ORIGINAL UPB,ORIGINAL LOAN TERM,ORIGINATION DATE,FIRST PAYMENT DATE,ORIGINAL LOAN-TO-VALUE (LTV),ORIGINAL COMBINED LOAN-TO-VALUE (CLTV),...,PROPERTY TYPE,NUMBER OF UNITS,OCCUPANCY TYPE,PROPERTY STATE,ZIP CODE SHORT,PRIMARY MORTGAGE INSURANCE PERCENT,PRODUCT TYPE,CO-BORROWER CREDIT SCORE AT ORIGINATION,MORTGAGE INSURANCE TYPE,RELOCATION MORTGAGE INDICATOR
0,100000102115,C,"CITIMORTGAGE, INC.",4.500,95000,120,02/2005,04/2005,51,51.0,...,SF,1,P,TX,750,NaN,FRM,783.0,NaN,N
1,100001329224,B,"BANK OF AMERICA, N.A.",5.875,141000,360,12/2004,02/2005,78,78.0,...,SF,1,P,IL,606,NaN,FRM,733.0,NaN,N
2,100004036273,C,"BANK OF AMERICA, N.A.",6.625,139000,360,01/2005,03/2005,95,95.0,...,SF,1,S,FL,349,30.0,FRM,NaN,1.0,N
3,100004945019,R,"BANK OF AMERICA, N.A.",5.875,104000,360,12/2004,02/2005,68,68.0,...,PU,1,P,SC,295,NaN,FRM,NaN,NaN,N
4,100013634177,C,"CITIMORTGAGE, INC.",6.250,51000,240,12/2004,02/2005,69,69.0,...,SF,1,P,WI,544,NaN,FRM,NaN,NaN,N


In [6]:
df_acq.describe()

,LOAN IDENTIFIER,ORIGINAL INTEREST RATE,ORIGINAL UPB,ORIGINAL LOAN TERM,ORIGINAL LOAN-TO-VALUE (LTV),ORIGINAL COMBINED LOAN-TO-VALUE (CLTV),NUMBER OF BORROWERS,ORIGINAL DEBT TO INCOME RATIO,BORROWER CREDIT SCORE AT ORIGINATION,NUMBER OF UNITS,ZIP CODE SHORT,PRIMARY MORTGAGE INSURANCE PERCENT,CO-BORROWER CREDIT SCORE AT ORIGINATION,MORTGAGE INSURANCE TYPE
count,3.036210e+05,303621.000000,303621.000000,303621.000000,303621.000000,303605.000000,303614.000000,282264.000000,301180.000000,303621.000000,303621.000000,40476.000000,145381.000000,40476.000000
mean,5.497815e+11,5.674627,162760.751068,311.845834,68.337019,69.981634,1.585787,36.042467,723.140836,1.039151,510.137994,22.689791,730.864315,1.041037
std,2.597903e+11,0.369377,83214.594649,79.046352,17.460744,18.134376,0.506746,13.160504,58.338406,0.247905,304.250841,7.225863,57.286263,0.198377
min,1.000001e+11,2.990000,7000.000000,60.000000,1.000000,1.000000,1.000000,1.000000,333.000000,1.000000,0.000000,6.000000,300.000000,1.000000
25%,3.250128e+11,5.500000,99000.000000,240.000000,58.000000,59.000000,1.000000,26.000000,680.000000,1.000000,275.000000,17.000000,691.000000,1.000000
50%,5.497583e+11,5.750000,147000.000000,360.000000,73.000000,74.000000,2.000000,36.000000,730.000000,1.000000,484.000000,25.000000,741.000000,1.000000
75%,7.748965e+11,5.875000,214000.000000,360.000000,80.000000,80.000000,2.000000,46.000000,773.000000,1.000000,800.000000,25.000000,779.000000,1.000000
max,9.999962e+11,9.000000,692000.000000,360.000000,97.000000,193.000000,7.000000,64.000000,850.000000,4.000000,999.000000,40.000000,842.000000,2.000000


In [7]:
df_acq.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303621 entries, 0 to 303620
Data columns (total 25 columns):
LOAN IDENTIFIER                            303621 non-null int64
ORIGINATION CHANNEL                        303621 non-null object
SELLER NAME                                303621 non-null object
ORIGINAL INTEREST RATE                     303621 non-null float64
ORIGINAL UPB                               303621 non-null int64
ORIGINAL LOAN TERM                         303621 non-null int64
ORIGINATION DATE                           303621 non-null object
FIRST PAYMENT DATE                         303621 non-null object
ORIGINAL LOAN-TO-VALUE (LTV)               303621 non-null int64
ORIGINAL COMBINED LOAN-TO-VALUE (CLTV)     303605 non-null float64
NUMBER OF BORROWERS                        303614 non-null float64
ORIGINAL DEBT TO INCOME RATIO              282264 non-null float64
BORROWER CREDIT SCORE AT ORIGINATION       301180 non-null float64
FIRST TIME HOME BUYER INDICATO

In [8]:
df_acq.isna().sum()

LOAN IDENTIFIER                                 0
ORIGINATION CHANNEL                             0
SELLER NAME                                     0
ORIGINAL INTEREST RATE                          0
ORIGINAL UPB                                    0
ORIGINAL LOAN TERM                              0
ORIGINATION DATE                                0
FIRST PAYMENT DATE                              0
ORIGINAL LOAN-TO-VALUE (LTV)                    0
ORIGINAL COMBINED LOAN-TO-VALUE (CLTV)         16
NUMBER OF BORROWERS                             7
ORIGINAL DEBT TO INCOME RATIO               21357
BORROWER CREDIT SCORE AT ORIGINATION         2441
FIRST TIME HOME BUYER INDICATOR                 0
LOAN PURPOSE                                    0
PROPERTY TYPE                                   0
NUMBER OF UNITS                                 0
OCCUPANCY TYPE                                  0
PROPERTY STATE                                  0
ZIP CODE SHORT                                  0


### Explore Performance File

In [9]:
df_perf.head()

,LOAN IDENTIFIER,MONTHLY REPORTING PERIOD,SERVICER NAME,CURRENT INTEREST RATE,CURRENT ACTUAL UPB,LOAN AGE,REMAINING MONTHS TO LEGAL MATURITY,ADJUSTED MONTHS TO MATURITY,MATURITY DATE,METROPOLITAN STATISTICAL AREA (MSA),...,ASSOCIATED TAXES FOR HOLDING PROPERTY,NET SALE PROCEEDS,CREDIT ENHANCEMENT PROCEEDS,REPURCHASE MAKE WHOLE PROCEEDS,OTHER FORECLOSURE PROCEEDS,NON INTEREST BEARING UPB,PRINCIPAL FORGIVENESS AMOUNT,REPURCHASE MAKE WHOLE PROCEEDS FLAG,FORECLOSURE PRINCIPAL WRITE-OFF AMOUNT,SERVICING ACTIVITY INDICATOR
0,100000102115,03/01/2005,"CITIMORTGAGE ASSET MANAGEMENT, INC.",4.5,NaN,0,120,119.0,03/2015,43300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100000102115,04/01/2005,NaN,4.5,NaN,1,119,118.0,03/2015,43300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100000102115,05/01/2005,NaN,4.5,NaN,2,118,117.0,03/2015,43300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100000102115,06/01/2005,NaN,4.5,NaN,3,117,116.0,03/2015,43300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100000102115,07/01/2005,NaN,4.5,NaN,4,116,115.0,03/2015,43300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
df_perf.describe()

,LOAN IDENTIFIER,CURRENT INTEREST RATE,CURRENT ACTUAL UPB,LOAN AGE,REMAINING MONTHS TO LEGAL MATURITY,ADJUSTED MONTHS TO MATURITY,METROPOLITAN STATISTICAL AREA (MSA),ZERO BALANCE CODE,FORECLOSURE COSTS,PROPERTY PRESERVATION AND REPAIR COSTS,ASSET RECOVERY COSTS,MISCELLANEOUS HOLDING EXPENSES AND CREDITS,ASSOCIATED TAXES FOR HOLDING PROPERTY,NET SALE PROCEEDS,CREDIT ENHANCEMENT PROCEEDS,REPURCHASE MAKE WHOLE PROCEEDS,OTHER FORECLOSURE PROCEEDS,NON INTEREST BEARING UPB,PRINCIPAL FORGIVENESS AMOUNT,FORECLOSURE PRINCIPAL WRITE-OFF AMOUNT
count,2.328899e+07,2.328899e+07,2.149180e+07,2.328899e+07,2.328899e+07,2.264002e+07,2.328899e+07,272317.000000,11864.000000,9388.000000,6198.000000,10992.000000,10608.000000,11851.000000,2671.000000,348.000000,6930.000000,632260.000000,8090.000000,133.000000
mean,5.501970e+11,5.605827e+00,1.366865e+05,5.347658e+01,2.599976e+02,2.474848e+02,2.648135e+04,1.333574,5391.553275,5887.109608,1594.178780,1841.153622,5843.308585,108271.997830,37978.179603,87660.840230,4870.120170,5364.168564,1100.046211,13483.769925
std,2.598858e+11,5.275908e-01,8.002704e+04,3.999058e+01,9.185173e+01,9.648315e+01,1.442334e+04,1.651240,6289.836398,7599.043698,1438.300249,4724.715245,8471.819727,74764.278957,27726.937753,74269.437655,17363.425169,17151.498695,7923.821481,8033.150590
min,1.000001e+11,2.000000e+00,0.000000e+00,-1.000000e+00,-4.300000e+01,0.000000e+00,0.000000e+00,1.000000,1.200000,1.500000,0.000000,-65461.830000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1881.420000
25%,3.253915e+11,5.375000e+00,7.513707e+04,2.100000e+01,1.890000e+02,1.710000e+02,1.674000e+04,1.000000,2649.420000,1815.000000,645.120000,-124.757500,1592.067500,49693.525000,19784.065000,36642.157500,305.000000,0.000000,0.000000,3652.230000
50%,5.504488e+11,5.750000e+00,1.209646e+05,4.500000e+01,2.920000e+02,2.830000e+02,2.982000e+04,1.000000,4112.710000,3082.190000,1394.750000,671.090000,3212.255000,96098.440000,32201.320000,81668.190000,1119.650000,0.000000,0.000000,13982.640000
75%,7.755155e+11,5.875000e+00,1.848676e+05,7.800000e+01,3.320000e+02,3.270000e+02,3.806000e+04,1.000000,6524.717500,7321.000000,2000.000000,2290.160000,6595.512500,152099.285000,48906.235000,127445.067500,3606.870000,0.000000,0.000000,20714.340000
max,9.999962e+11,9.000000e+00,8.925631e+05,2.230000e+02,4.840000e+02,4.790000e+02,4.974000e+04,16.000000,292354.110000,255658.170000,28358.500000,71802.170000,207520.140000,606352.200000,361688.770000,571955.760000,358122.120000,246677.850000,107025.180000,24204.640000


In [11]:
df_perf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23288993 entries, 0 to 23288992
Data columns (total 31 columns):
LOAN IDENTIFIER                               int64
MONTHLY REPORTING PERIOD                      object
SERVICER NAME                                 object
CURRENT INTEREST RATE                         float64
CURRENT ACTUAL UPB                            float64
LOAN AGE                                      int64
REMAINING MONTHS TO LEGAL MATURITY            int64
ADJUSTED MONTHS TO MATURITY                   float64
MATURITY DATE                                 object
METROPOLITAN STATISTICAL AREA (MSA)           int64
CURRENT LOAN DELINQUENCY STATUS               object
MODIFICATION FLAG                             object
ZERO BALANCE CODE                             float64
ZERO BALANCE EFFECTIVE DATE                   object
LAST PAID INSTALLMENT DATE                    object
FORECLOSURE DATE                              object
DISPOSITION DATE                     

In [12]:
df_perf.isna().sum()

LOAN IDENTIFIER                                      0
MONTHLY REPORTING PERIOD                             0
SERVICER NAME                                 22716526
CURRENT INTEREST RATE                                0
CURRENT ACTUAL UPB                             1797191
LOAN AGE                                             0
REMAINING MONTHS TO LEGAL MATURITY                   0
ADJUSTED MONTHS TO MATURITY                     648976
MATURITY DATE                                        0
METROPOLITAN STATISTICAL AREA (MSA)                  0
CURRENT LOAN DELINQUENCY STATUS                  12325
MODIFICATION FLAG                                    0
ZERO BALANCE CODE                             23016676
ZERO BALANCE EFFECTIVE DATE                   23016676
LAST PAID INSTALLMENT DATE                    23276668
FORECLOSURE DATE                              23277275
DISPOSITION DATE                              23276890
FORECLOSURE COSTS                             23277129
PROPERTY P

# add write up on above
- what you see in the data
- what will you do about nulls, columns, etc.

In [15]:
# define function to clean and merge datasets
### needs write up on assumptions and decisions on each step

def clean_merge_datasets(df_acq=df_acq, df_perf=df_perf, year=year, quarter=quarter):

    # condense df_perf down to last status of each loan
    loan_ids = df_perf['LOAN IDENTIFIER'].tolist()

    last_index = []
    for i in range(1,len(loan_ids)):
        if loan_ids[i] != loan_ids[i-1]:
            last_index.append(i-1)

        if i == len(loan_ids)-1:
            last_index.append(i)

    df_perf = df_perf.iloc[last_index]

    # condense df_perf down to loan id and zero balance code
    df_perf = df_perf[['LOAN IDENTIFIER', 'ZERO BALANCE CODE']]

    # rename target column to default
    df_perf.rename({'ZERO BALANCE CODE': 'DEFAULT'}, axis=1, inplace=True)

    # map zero balance codes to binary 
    df_perf['DEFAULT'] = df_perf['DEFAULT'].map(lambda x: 1 if x in [2, 3, 6, 9, 15] else 0)

    # merge
    df_cmp = pd.merge(df_acq, df_perf, on='LOAN IDENTIFIER')

    # create binary MI column
    df_cmp['MI'] = df_cmp['PRIMARY MORTGAGE INSURANCE PERCENT'].fillna(0).map(lambda x: 1 if x > 0 else x)

    # drop nulls
    df_cmp.dropna(subset=['DEFAULT', 'ORIGINAL COMBINED LOAN-TO-VALUE (CLTV)', 'NUMBER OF BORROWERS', 'ORIGINAL DEBT TO INCOME RATIO', 
                        'BORROWER CREDIT SCORE AT ORIGINATION'], inplace=True)

    # create MINIMUM CREDIT SCORE column
    df_cmp['CO-BORROWER CREDIT SCORE AT ORIGINATION'].fillna(1000, inplace=True)
    df_cmp['MIN CREDIT SCORE'] = df_cmp[['BORROWER CREDIT SCORE AT ORIGINATION', 'CO-BORROWER CREDIT SCORE AT ORIGINATION']].min(axis=1)

    # drop columns
    df_cmp.drop(columns=['PRIMARY MORTGAGE INSURANCE PERCENT', 'MORTGAGE INSURANCE TYPE', 
                       'BORROWER CREDIT SCORE AT ORIGINATION', 'CO-BORROWER CREDIT SCORE AT ORIGINATION'], inplace=True)

    # reset index
    df_cmp.reset_index(drop=True, inplace=True)

    # save to csv
    df_cmp.to_csv(f'./data/complete{year}q{quarter}.csv', index=False)
    
    return df_cmp

In [16]:
df_cmp = clean_merge_datasets()

In [17]:
df_cmp.isna().sum()

LOAN IDENTIFIER                           0
ORIGINATION CHANNEL                       0
SELLER NAME                               0
ORIGINAL INTEREST RATE                    0
ORIGINAL UPB                              0
ORIGINAL LOAN TERM                        0
ORIGINATION DATE                          0
FIRST PAYMENT DATE                        0
ORIGINAL LOAN-TO-VALUE (LTV)              0
ORIGINAL COMBINED LOAN-TO-VALUE (CLTV)    0
NUMBER OF BORROWERS                       0
ORIGINAL DEBT TO INCOME RATIO             0
FIRST TIME HOME BUYER INDICATOR           0
LOAN PURPOSE                              0
PROPERTY TYPE                             0
NUMBER OF UNITS                           0
OCCUPANCY TYPE                            0
PROPERTY STATE                            0
ZIP CODE SHORT                            0
PRODUCT TYPE                              0
RELOCATION MORTGAGE INDICATOR             0
DEFAULT                                   0
MI                              

In [18]:
df_cmp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 279961 entries, 0 to 279960
Data columns (total 24 columns):
LOAN IDENTIFIER                           279961 non-null int64
ORIGINATION CHANNEL                       279961 non-null object
SELLER NAME                               279961 non-null object
ORIGINAL INTEREST RATE                    279961 non-null float64
ORIGINAL UPB                              279961 non-null int64
ORIGINAL LOAN TERM                        279961 non-null int64
ORIGINATION DATE                          279961 non-null object
FIRST PAYMENT DATE                        279961 non-null object
ORIGINAL LOAN-TO-VALUE (LTV)              279961 non-null int64
ORIGINAL COMBINED LOAN-TO-VALUE (CLTV)    279961 non-null float64
NUMBER OF BORROWERS                       279961 non-null float64
ORIGINAL DEBT TO INCOME RATIO             279961 non-null float64
FIRST TIME HOME BUYER INDICATOR           279961 non-null object
LOAN PURPOSE                              27